# Explaining functions and parameters from busi_preprocessing_1

## def remove_annotations(self, img)

### 1. _, mask = cv2.threshold(img, 220, 255, cv2.THRESH_BINARY) 
  - applies binary thresholding to detect very bright pixels (e.g., white annotations).

- Parameters:

img: the input grayscale image.

220: threshold value. Pixels with value >220 will be set to 255(white), others to 0(black).

255: max value to assign to thresholded pixels.

cv2.THRESH_BINARY: sets all pixels above the threshold to 255, the rest to 0.

- Output:

_: the actual threshold used(not needed here).

mask: a binary mask where white regions (255) indicate possible annotations.

- Effect:

Creates a binary map of potential annotation regions.

### 2. kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

- defines a morphological kernel for processing the mask.

- Parameters:

cv2.MORPH_RECT: defines a rectangular kernel shape.

(3, 3): kernel size: a 3×3 square

- Effect:

The kernel will be used in dilation to expand white regions (i.e., annotation marks) a bit, making them more detectable for inpainting.

### 3. mask = cv2.dilate(mask, kernel, iterations=1)

- Applies dilation to the binary mask

- Parameters:

mask: input binary mask from thresholding

kernel: the structuring element from the previous step

iterations=1: how many times to apply dilation.

- Effect:

Slightly enlarges the white regions, allowing thin text or lines (like crosses) to be more effectively covered and removed.

### 4. inpainted = cv2.inpaint(img, mask, inpaintRadius=1, flags=cv2.INPAINT_TELEA)

- Performs image inpainting, i.e., reconstructs regions defined by the mask using surrounding pixels.

- Parameters:

img: original grayscale image

mask: binary mask — white areas are inpainted

inpaintRadius=1: defines how far out to look around each pixel to reconstruct it (small values (like 1–3) are usually enough for thin annotations)

flags=cv2.INPAINT_TELEA: uses the TELEA algorithm, a fast inpainting method good for small, sharp features

- Effect:

Reconstructs annotated areas using nearby pixel information to make them "disappear".

## def enhance_contrast(self, img)

### 1. img = (img * 255).astype(np.uint8)
If the image is in float format (e.g., values in range [0, 1]), it’s scaled to [0, 255] and converted to 8-bit unsigned integers; CLAHE in OpenCV only works with uint8 images

### 2. clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

- This creates a CLAHE object with specific parameters.

*CLAHE = Contrast Limited Adaptive Histogram Equalization* - a localized form of histogram equalization — great for preserving detail and avoiding over-amplification of noise in ultrasound images.

- Parameters:

      -- clipLimit=2.0:

Prevents noise amplification by limiting the contrast enhancement in very uniform regions.

Typical values: 1.0–4.0; smaller = less enhancement.

     -- tileGridSize=(8, 8):

Image is divided into 8×8 tiles (regions).

CLAHE is applied separately to each tile → preserves local contrast better than global equalization.